Proyek ini bertujuan untuk melakukan **web scraping** dengan memanfaatkan **BeautifulSoup** pada data **Daftar Penduduk Potensial Pemilih Pemilu (DP4) dari KPU**, yang mencakup jumlah pemilih per provinsi (tidak sampai pada kota/kab dan kecamatan) dan proporsi gender. Data yang diperoleh digunakan untuk **menghitung jumlah sampel** menggunakan rumus **Slovin** dengan Margin of Error 3%. kemudian diterapkan **stratified random sampling** berdasarkan provinsi dan gender. Untuk pemilihan sample sendiri, dengan memakai numpy, randomisasi dilakukan berdasarkan nomor tertentu di range total jumlah perempuan dan laki-laki.

Sebagai eksperimen, oversampling diterapkan pada tujuh provinsi dengan sampel kecil untuk memperbesar ukuran sampel (dikalikan faktor 2). Setelah itu, pembobotan dihitung untuk memastikan representasi yang akurat dalam hasil analisis. Proyek ini bertujuan untuk memastikan distribusi sampel yang seimbang dan akurat, meskipun ada ketidakseimbangan dalam jumlah pemilih antarprovinsi.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# Daftar kode provinsi yang valid
kode_provinsi_valid = [
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    21, 31, 32, 33, 34, 35, 36, 51, 52, 53, 61, 62, 63, 64, 65, 71, 72,
    73, 74, 75, 76, 81, 82, 91, 92
]

# Inisialisasi list untuk menyimpan data hasil scraping
data = []

# Looping untuk setiap kode provinsi
for kode in kode_provinsi_valid:
    url = f'https://satupetadata.kpu.go.id/data_pemilih_prov/{kode}'
    try:
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            kotakkanan = soup.find(id='kotakkanan')

            if kotakkanan:
                try:
                    # Ambil nama provinsi
                    provinsi_element = kotakkanan.find('b', string='Provinsi')
                    provinsi = (
                        provinsi_element.find_next('br').next_sibling.strip()
                        if provinsi_element else 'Tidak Ditemukan'
                    )

                    # Cari tag <script> yang berisi data jumlah pemilih
                    script_tag = kotakkanan.find('script', type='text/javascript')

                    jumlah_laki_laki = 'Tidak Ditemukan'
                    jumlah_perempuan = 'Tidak Ditemukan'
                    jumlah_total = 'Tidak Ditemukan'

                    if script_tag and script_tag.string:
                        script_content = script_tag.string

                        # Cari data jumlah pemilih laki-laki dan perempuan dengan regex
                        laki_laki_match = re.search(r"'Laki-laki',\s*([\d\.]+)", script_content)
                        perempuan_match = re.search(r"'Perempuan',\s*([\d\.]+)", script_content)

                        if laki_laki_match:
                            jumlah_laki_laki = laki_laki_match.group(1).replace('.', '')
                        if perempuan_match:
                            jumlah_perempuan = perempuan_match.group(1).replace('.', '')

                        # Hitung total jika data valid
                        if jumlah_laki_laki != 'Tidak Ditemukan' and jumlah_perempuan != 'Tidak Ditemukan':
                            try:
                                jumlah_total = int(jumlah_laki_laki) + int(jumlah_perempuan)
                            except ValueError:
                                jumlah_total = 'Tidak Dapat Dihitung'

                    # Simpan data ke list
                    data.append({
                        'Kode Provinsi': kode,
                        'Provinsi': provinsi,
                        'Jumlah Laki-laki': jumlah_laki_laki,
                        'Jumlah Perempuan': jumlah_perempuan,
                        'Jumlah Total': jumlah_total
                    })

                    print(f'Berhasil mengambil data dari Provinsi {provinsi} (Kode: {kode})')

                except AttributeError as e:
                    print(f'Gagal mengambil data dari Kode Provinsi {kode} karena struktur HTML tidak sesuai: {e}')
            else:
                print(f"Elemen dengan id 'kotakkanan' tidak ditemukan pada Kode Provinsi {kode}.")

        elif response.status_code == 404:
            print(f"Halaman tidak ditemukan untuk Kode Provinsi {kode}. Status code: {response.status_code}")

        elif response.status_code == 500:
            print(f"Server error saat mengambil halaman untuk Kode Provinsi {kode}. Status code: {response.status_code}")

        else:
            print(f"Gagal mengambil halaman untuk Kode Provinsi {kode}. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan koneksi atau timeout untuk Kode Provinsi {kode}: {e}")

    # Delay 1 detik agar tidak membebani server
    time.sleep(1)

# Buat DataFrame dari data yang terkumpul
df = pd.DataFrame(data)

# Simpan data ke file CSV
df.to_csv('data_pemilih_provinsi.csv', index=False, encoding='utf-8')

print("Data telah disimpan ke dalam file 'data_pemilih_provinsi.csv'")


Berhasil mengambil data dari Provinsi ACEH (Kode: 11)
Berhasil mengambil data dari Provinsi SUMATERA UTARA (Kode: 12)
Berhasil mengambil data dari Provinsi SUMATERA BARAT (Kode: 13)
Berhasil mengambil data dari Provinsi RIAU (Kode: 14)
Berhasil mengambil data dari Provinsi JAMBI (Kode: 15)
Berhasil mengambil data dari Provinsi SUMATERA SELATAN (Kode: 16)
Berhasil mengambil data dari Provinsi BENGKULU (Kode: 17)
Berhasil mengambil data dari Provinsi LAMPUNG (Kode: 18)
Berhasil mengambil data dari Provinsi KEPULAUAN BANGKA BELITUNG (Kode: 19)
Server error saat mengambil halaman untuk Kode Provinsi 20. Status code: 500
Berhasil mengambil data dari Provinsi KEPULAUAN RIAU (Kode: 21)
Berhasil mengambil data dari Provinsi DKI JAKARTA (Kode: 31)
Berhasil mengambil data dari Provinsi JAWA BARAT (Kode: 32)
Berhasil mengambil data dari Provinsi JAWA TENGAH (Kode: 33)
Berhasil mengambil data dari Provinsi DAERAH ISTIMEWA YOGYAKARTA (Kode: 34)
Berhasil mengambil data dari Provinsi JAWA TIMUR (Kode

In [2]:
df

,Kode Provinsi,Provinsi,Jumlah Laki-laki,Jumlah Perempuan,Jumlah Total
0,11,ACEH,1837771,1914365,3752136
1,12,SUMATERA UTARA,5494976,5628260,11123236
2,13,SUMATERA BARAT,2039559,2086584,4126143
3,14,RIAU,2401972,2338420,4740392
4,15,JAMBI,1343947,1321703,2665650
5,16,SUMATERA SELATAN,3179997,3121808,6301805
6,17,BENGKULU,759107,744053,1503160
7,18,LAMPUNG,3328268,3203693,6531961
8,19,KEPULAUAN BANGKA BELITUNG,541814,525182,1066996
9,21,KEPULAUAN RIAU,753999,745950,1499949


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Kode Provinsi     34 non-null     int64 
 1   Provinsi          34 non-null     object
 2   Jumlah Laki-laki  34 non-null     object
 3   Jumlah Perempuan  34 non-null     object
 4   Jumlah Total      34 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 1.5+ KB


In [4]:
df['Jumlah Laki-laki'] = pd.to_numeric(df["Jumlah Laki-laki"], errors='coerce')
df["Jumlah Perempuan"] = pd.to_numeric(df["Jumlah Perempuan"], errors='coerce')


## Stratified Random Sampling

**Rumus Ukuran Sampel Menggunakan Slovin**

Rumus untuk menghitung ukuran sampel (n) berdasarkan **Slovin's Formula** adalah:


n = N / (1 + N * e^2)

Dimana:
- n = ukuran sampel yang dihitung
- N = total populasi
- e = margin of error (dalam bentuk desimal, misalnya 0.03 untuk 3%)

**Proporsi Sampel per Provinsi**

Setelah ukuran sampel dihitung, proporsi sampel untuk setiap provinsi dapat dihitung dengan rumus berikut:

Proporsi Sampel = (Jumlah Total Pemilih Provinsi / Total Populasi) * n

Dimana:
- Jumlah Total Pemilih Provinsi = jumlah pemilih di provinsi tertentu
- Total Populasi = jumlah pemilih keseluruhan
- n = ukuran sampel yang dihitung sebelumnya

**Pembagian Sampel Berdasarkan Gender**

Untuk menghitung jumlah sampel laki-laki dan perempuan berdasarkan proporsi sampel per provinsi, rumusnya adalah:

Sampel Laki-laki = (Jumlah Laki-laki / Jumlah Total Pemilih) * Proporsi Sampel

Sampel Perempuan = (Jumlah Perempuan / Jumlah Total Pemilih) * Proporsi Sampel

Dimana:
- Jumlah Laki-laki dan Jumlah Perempuan adalah jumlah pemilih laki-laki dan perempuan di provinsi.
- Proporsi Sampel adalah proporsi sampel yang dihitung sebelumnya.

Setelah perhitungan ini, hasil sampel dibulatkan ke angka bulat menggunakan fungsi pembulatan untuk memastikan nilai sampel adalah integer.


In [5]:
# Total populasi
total_populasi = df["Jumlah Total"].sum()

# Margin of error (3%)
e = 0.03

# Menghitung ukuran sampel menggunakan rumus Slovin
n = total_populasi / (1 + total_populasi * e**2)
n = round(n) # Membulatkan hasil

print(f"Total populasi: {total_populasi}")
print(f"Ukuran sampel yang dihitung dengan margin of error {e*100}%: {n}")

# Menghitung proporsi untuk setiap provinsi
df["Proporsi Sampel"] = ((df["Jumlah Total"] / total_populasi) * n).round(2)

# Menghitung jumlah sampel laki-laki dan perempuan berdasarkan proporsi
df["Sampel Laki-laki"] = (df["Jumlah Laki-laki"] / df["Jumlah Total"]) * df["Proporsi Sampel"]
df["Sampel Perempuan"] = (df["Jumlah Perempuan"] / df["Jumlah Total"]) * df["Proporsi Sampel"]

# Mengonversi ke integer (jumlah sampel harus bulat)
df["Sampel Laki-laki"] = df["Sampel Laki-laki"].round().astype(int)
df["Sampel Perempuan"] = df["Sampel Perempuan"].round().astype(int)

Total populasi: 201507007
Ukuran sampel yang dihitung dengan margin of error 3.0%: 1111


In [6]:
df.head()

,Kode Provinsi,Provinsi,Jumlah Laki-laki,Jumlah Perempuan,Jumlah Total,Proporsi Sampel,Sampel Laki-laki,Sampel Perempuan
0,11,ACEH,1837771,1914365,3752136,20.69,10,11
1,12,SUMATERA UTARA,5494976,5628260,11123236,61.33,30,31
2,13,SUMATERA BARAT,2039559,2086584,4126143,22.75,11,12
3,14,RIAU,2401972,2338420,4740392,26.14,13,13
4,15,JAMBI,1343947,1321703,2665650,14.70,7,7


In [29]:
hasil_sampel_df = df[["Kode Provinsi", "Provinsi", "Sampel Laki-laki", "Sampel Perempuan"]]
hasil_sampel_df

,Kode Provinsi,Provinsi,Sampel Laki-laki,Sampel Perempuan
0,11,ACEH,10,11
1,12,SUMATERA UTARA,30,31
2,13,SUMATERA BARAT,11,12
3,14,RIAU,13,13
4,15,JAMBI,7,7
5,16,SUMATERA SELATAN,18,17
6,17,BENGKULU,4,4
7,18,LAMPUNG,18,18
8,19,KEPULAUAN BANGKA BELITUNG,3,3
9,21,KEPULAUAN RIAU,4,4


In [9]:
df.sort_values(by='Jumlah Total', ascending=True)

,Kode Provinsi,Provinsi,Jumlah Laki-laki,Jumlah Perempuan,Jumlah Total,Proporsi Sampel,Sampel Laki-laki,Sampel Perempuan
33,92,PAPUA BARAT,199043,191171,390214,2.15,1,1
23,65,KALIMANTAN UTARA,258472,238830,497302,2.74,1,1
32,91,P A P U A,373435,358843,732278,4.04,2,2
28,75,GORONTALO,442042,447565,889607,4.90,2,2
31,82,MALUKU UTARA,503760,481113,984873,5.43,3,3
29,76,SULAWESI BARAT,517827,513659,1031486,5.69,3,3
8,19,KEPULAUAN BANGKA BELITUNG,541814,525182,1066996,5.88,3,3
30,81,MALUKU,658857,682206,1341063,7.39,4,4
9,21,KEPULAUAN RIAU,753999,745950,1499949,8.27,4,4
6,17,BENGKULU,759107,744053,1503160,8.29,4,4


## Simulasi mengambil sampel acak 

In [10]:
import numpy as np

In [11]:
def ambil_sampel(row):
    laki_laki_terpilih = np.random.choice(np.arange(row["Jumlah Laki-laki"]), 
                                          size=row["Sampel Laki-laki"], 
                                          replace=False)
    perempuan_terpilih = np.random.choice(np.arange(row["Jumlah Perempuan"]), 
                                          size=row["Sampel Perempuan"], 
                                          replace=False)
    return {"Laki-laki": laki_laki_terpilih.tolist(), "Perempuan": perempuan_terpilih.tolist()}


In [12]:
sampel_individu = df.apply(ambil_sampel, axis=1)


In [13]:
# Membuat DataFrame untuk hasil sampel individu
sampel_individu_df = pd.DataFrame(sampel_individu.tolist(), index=df.index)
sampel_individu_df['Kode Provinsi'] = df['Kode Provinsi']
sampel_individu_df['Provinsi'] = df['Provinsi']

In [14]:
sampel_individu_df

,Laki-laki,Perempuan,Kode Provinsi,Provinsi
0,"[1374813, 386934, 191604, 725552, 178792, 3178...","[714822, 796888, 1469865, 1532745, 254708, 486...",11,ACEH
1,"[2371897, 1822394, 1008969, 1011581, 4283951, ...","[1514608, 2830175, 627892, 2230012, 279015, 30...",12,SUMATERA UTARA
2,"[546042, 11081, 2009114, 1425003, 1403378, 445...","[471358, 868013, 662263, 72404, 2035292, 68520...",13,SUMATERA BARAT
3,"[114379, 2234270, 1119486, 1183218, 937276, 51...","[1441093, 854487, 313222, 1215365, 2062945, 75...",14,RIAU
4,"[660254, 726489, 720531, 278840, 941975, 78695...","[467940, 691241, 1126787, 17362, 51033, 577798...",15,JAMBI
5,"[2155679, 2850713, 199961, 313904, 2558087, 11...","[1560453, 897624, 91483, 3109533, 2668307, 818...",16,SUMATERA SELATAN
6,"[420911, 698675, 6105, 396370]","[95645, 500794, 216832, 598037]",17,BENGKULU
7,"[3302048, 3234404, 1621856, 382614, 1894211, 1...","[1471055, 362943, 2321613, 497860, 836721, 791...",18,LAMPUNG
8,"[521013, 504177, 286839]","[12917, 289788, 19737]",19,KEPULAUAN BANGKA BELITUNG
9,"[169358, 164542, 615849, 359095]","[128309, 99738, 407565, 542868]",21,KEPULAUAN RIAU


### Oversampling

In [15]:
# 1. Identifikasi 7 Provinsi dengan Jumlah Populasi Total Terbawah dari df
tujuh_terbawah_pop_df = df.sort_values(by='Jumlah Total').head(7)['Kode Provinsi'].tolist()
print("7 Kode Provinsi dengan Jumlah Populasi Total Terbawah:", tujuh_terbawah_pop_df)

7 Kode Provinsi dengan Jumlah Populasi Total Terbawah: [92, 65, 91, 75, 82, 76, 19]


In [16]:
# 2. Tentukan Faktor Oversampling
faktor_oversampling = 2

In [17]:
# 3. Lakukan Oversampling pada hasil_sampel_df
hasil_sampel_df_oversampled = hasil_sampel_df.copy()

mask = hasil_sampel_df_oversampled['Kode Provinsi'].isin(tujuh_terbawah_pop_df)
hasil_sampel_df_oversampled.loc[mask, 'Sampel Laki-laki'] *= faktor_oversampling
hasil_sampel_df_oversampled.loc[mask, 'Sampel Perempuan'] *= faktor_oversampling

print("\nDataFrame hasil_sampel_df setelah oversampling:")
hasil_sampel_df_oversampled


DataFrame hasil_sampel_df setelah oversampling:


,Kode Provinsi,Provinsi,Sampel Laki-laki,Sampel Perempuan
0,11,ACEH,10,11
1,12,SUMATERA UTARA,30,31
2,13,SUMATERA BARAT,11,12
3,14,RIAU,13,13
4,15,JAMBI,7,7
5,16,SUMATERA SELATAN,18,17
6,17,BENGKULU,4,4
7,18,LAMPUNG,18,18
8,19,KEPULAUAN BANGKA BELITUNG,6,6
9,21,KEPULAUAN RIAU,4,4


In [18]:
# 4. Fungsi untuk mengambil sampel individu
def ambil_sampel_oversampled_v2(row, df_populasi):
    provinsi_data = df_populasi[df_populasi['Kode Provinsi'] == row['Kode Provinsi']].iloc[0]
    jumlah_laki_laki_pop = provinsi_data['Jumlah Laki-laki']
    jumlah_perempuan_pop = provinsi_data['Jumlah Perempuan']
    
    sampel_laki_laki_target = int(row['Sampel Laki-laki'])
    sampel_perempuan_target = int(row['Sampel Perempuan'])
    
    # Hindari sampling lebih besar dari populasi (replace=False tidak bisa jalan kalau sample > populasi)
    if sampel_laki_laki_target > jumlah_laki_laki_pop or sampel_perempuan_target > jumlah_perempuan_pop:
        raise ValueError(f"Jumlah sampel melebihi populasi untuk Kode Provinsi {row['Kode Provinsi']}")

    laki_laki_terpilih = np.random.choice(np.arange(jumlah_laki_laki_pop), size=sampel_laki_laki_target, replace=False).tolist()
    perempuan_terpilih = np.random.choice(np.arange(jumlah_perempuan_pop), size=sampel_perempuan_target, replace=False).tolist()

    return {"Laki-laki": laki_laki_terpilih, "Perempuan": perempuan_terpilih}


In [19]:
# 5. Terapkan fungsi sampling ke setiap baris
sampel_individu_oversampled = hasil_sampel_df_oversampled.apply(
    ambil_sampel_oversampled_v2, axis=1, df_populasi=df
)

In [20]:
# 6. Buat DataFrame dari hasil sampling
sampel_individu_oversampled_df = pd.DataFrame(
    sampel_individu_oversampled.tolist(), 
    index=hasil_sampel_df_oversampled.index
)

In [21]:
# 7. Tambahkan informasi provinsi
sampel_individu_oversampled_df['Kode Provinsi'] = hasil_sampel_df_oversampled['Kode Provinsi']
sampel_individu_oversampled_df['Provinsi'] = hasil_sampel_df_oversampled['Provinsi']


In [22]:
# 8.  Hasil
print("\nDataFrame Sampel Individu yang Terpilih (Oversampled):")
sampel_individu_oversampled_df[['Kode Provinsi', 'Provinsi', 'Laki-laki', 'Perempuan']]



DataFrame Sampel Individu yang Terpilih (Oversampled):


,Kode Provinsi,Provinsi,Laki-laki,Perempuan
0,11,ACEH,"[307809, 252592, 300976, 766620, 76013, 154245...","[872158, 1788613, 864467, 1381019, 584707, 111..."
1,12,SUMATERA UTARA,"[3601190, 1841606, 1426770, 2986685, 4163737, ...","[1832142, 3003428, 3354504, 3353595, 2659445, ..."
2,13,SUMATERA BARAT,"[1504891, 1074149, 600176, 1838562, 1564201, 3...","[1648665, 1788889, 751343, 1770789, 433616, 91..."
3,14,RIAU,"[445812, 1199636, 252410, 2171764, 363791, 130...","[1165877, 1924169, 421779, 1065825, 1836418, 1..."
4,15,JAMBI,"[592061, 1177869, 931526, 1238045, 1315114, 73...","[1311265, 66145, 965310, 900415, 116247, 87791..."
5,16,SUMATERA SELATAN,"[1953961, 583288, 2086726, 675962, 20097, 6337...","[2095269, 347397, 449423, 1604744, 1564535, 28..."
6,17,BENGKULU,"[513781, 230241, 403177, 570537]","[228120, 334038, 211466, 661512]"
7,18,LAMPUNG,"[3193723, 2205101, 1222903, 2041476, 2175234, ...","[1640966, 1167579, 762326, 1690621, 832293, 32..."
8,19,KEPULAUAN BANGKA BELITUNG,"[112660, 92436, 25590, 316511, 269219, 117866]","[320023, 340848, 193354, 307146, 17855, 362969]"
9,21,KEPULAUAN RIAU,"[266711, 177234, 407373, 422105]","[248413, 575996, 45978, 382527]"


### Bobot

In [23]:
df['Proporsi Populasi'] = df['Jumlah Total'] / df['Jumlah Total'].sum()

In [24]:
hasil_sampel_df_oversampled['Total Sampel'] = hasil_sampel_df_oversampled[['Sampel Laki-laki', 'Sampel Perempuan']].sum(axis=1)
hasil_sampel_df_oversampled['Proporsi Sampel'] = hasil_sampel_df_oversampled['Total Sampel'] / hasil_sampel_df_oversampled['Total Sampel'].sum()


In [25]:
# Gabungkan proporsi populasi dan proporsi sampel
df_bobot = hasil_sampel_df_oversampled.merge(
    df[['Kode Provinsi', 'Proporsi Populasi']], on='Kode Provinsi', how='left'
)

df_bobot['Bobot Koreksi'] = df_bobot['Proporsi Populasi'] / df_bobot['Proporsi Sampel']


In [26]:
df_bobot

,Kode Provinsi,Provinsi,Sampel Laki-laki,Sampel Perempuan,Total Sampel,Proporsi Sampel,Proporsi Populasi,Bobot Koreksi
0,11,ACEH,10,11,21,0.018453,0.018620,1.009047
1,12,SUMATERA UTARA,30,31,61,0.053603,0.055200,1.029801
2,13,SUMATERA BARAT,11,12,23,0.020211,0.020476,1.013138
3,14,RIAU,13,13,26,0.022847,0.023525,1.029658
4,15,JAMBI,7,7,14,0.012302,0.013229,1.075294
5,16,SUMATERA SELATAN,18,17,35,0.030756,0.031273,1.016832
6,17,BENGKULU,4,4,8,0.007030,0.007460,1.061127
7,18,LAMPUNG,18,18,36,0.031634,0.032416,1.024692
8,19,KEPULAUAN BANGKA BELITUNG,6,6,12,0.010545,0.005295,0.502150
9,21,KEPULAUAN RIAU,4,4,8,0.007030,0.007444,1.058860


In [27]:
df_individu = sampel_individu_oversampled_df.merge(
    df_bobot[['Kode Provinsi', 'Bobot Koreksi']],
    on='Kode Provinsi', how='left'
)


In [28]:
df_individu

,Laki-laki,Perempuan,Kode Provinsi,Provinsi,Bobot Koreksi
0,"[307809, 252592, 300976, 766620, 76013, 154245...","[872158, 1788613, 864467, 1381019, 584707, 111...",11,ACEH,1.009047
1,"[3601190, 1841606, 1426770, 2986685, 4163737, ...","[1832142, 3003428, 3354504, 3353595, 2659445, ...",12,SUMATERA UTARA,1.029801
2,"[1504891, 1074149, 600176, 1838562, 1564201, 3...","[1648665, 1788889, 751343, 1770789, 433616, 91...",13,SUMATERA BARAT,1.013138
3,"[445812, 1199636, 252410, 2171764, 363791, 130...","[1165877, 1924169, 421779, 1065825, 1836418, 1...",14,RIAU,1.029658
4,"[592061, 1177869, 931526, 1238045, 1315114, 73...","[1311265, 66145, 965310, 900415, 116247, 87791...",15,JAMBI,1.075294
5,"[1953961, 583288, 2086726, 675962, 20097, 6337...","[2095269, 347397, 449423, 1604744, 1564535, 28...",16,SUMATERA SELATAN,1.016832
6,"[513781, 230241, 403177, 570537]","[228120, 334038, 211466, 661512]",17,BENGKULU,1.061127
7,"[3193723, 2205101, 1222903, 2041476, 2175234, ...","[1640966, 1167579, 762326, 1690621, 832293, 32...",18,LAMPUNG,1.024692
8,"[112660, 92436, 25590, 316511, 269219, 117866]","[320023, 340848, 193354, 307146, 17855, 362969]",19,KEPULAUAN BANGKA BELITUNG,0.502150
9,"[266711, 177234, 407373, 422105]","[248413, 575996, 45978, 382527]",21,KEPULAUAN RIAU,1.058860
